# Goodreads
Progetto per il corso di Big Data, anno accademico 2021/22<br>
<br>
**Autore**: Yuqi Sun<br>
yuqi.sun@studio.unibo.it<br>
1005091

**Fonte**:<br>
*Mengting Wan*, *Julian McAuley*, "Item Recommendation on Monotonic Behavior Chains" (RecSys'18) <br>
*Mengting Wan*, *Rishabh Misra*, *Ndapa Nakashole*, *Julian McAuley*, "Fine-Grained Spoiler Detection from Large-Scale Review Corpora" (ACL'19)

## Descrizione del dataset
I dataset usati in questo progetto sono stati ottenuti dal seguente [sito](https://sites.google.com/eng.ucsd.edu/ucsdbookgraph) e memorizzano le informazioni riguardanti:
- i libri e le serie;
- gli autori;
- le interazioni tra utenti e libri.

Poiché il dataset riguardanti i libri e gli autori era in formato JSON, questi sono stati convertiti in JSON, risultando in un totale di circa 6GB di dati.

### Libri
Il file *goodreads_books.csv* contiene i seguenti dati:
- `book_id`: l'id del libro;
- `work_id`: l'id dell'edizione del libro;
- `title`: titolo del libro;
- `title_without_series`: titolo del libro senza il nome della serie;
- `description`: descrizione del libro;
- `series`: eventuale id della serie a cui appartiene il libro;
- `ratings_count`: numero di giudizi ottenuti;
- `average_rating`: giudizio medio;
- `similar_books`: lista di id dei libri simili;
- `country_code`: nazione del libro in codice (US, IT etc...);
- `language_code`: linguaggio del libro in codice (eng, ita...;
- `edition_information`: informazioni riguardanti l'edizione (Modern Library Classics, Penguin Classics...);
- `format`: formato dell'edizione (hardcover, paperback, kindle...);
- `is_ebook`: booleano indicante se è un ebook o meno;
- `num_pages`: numeor di pagine;
- `publication_day`: giorno di pubblicazione;
- `publication_month`: mese di pubblicazione;
- `publication_year`: anno di pubblicazione;
- `publisher`: casa editrice;
- `text_reviews_count`: numero di recensioni scritte.

Poiché in origine nel file JSON l'autore del libro era un array formato da due valori {`author_id`, `role`}, per semplicità è stato generato un ulteriore file *book_author_map.csv* che mappa da `book_id` a `author_id`, effettuando un'esplosione della struttura dati e selezionando solo l'id.

### Serie
Il file *goodreads_book_series.csv* contiene i seguenti campi:
- `series_id`: id della serie;
- `title`: titolo della serie;
- `description`: descrizione della serie;
- `note`: note;
- `numbered`: indica se la serie è numerata o meno;
- `primary_work_count`: numero di libri contenuti nella serie;
- `series_works_count`: numero totale di libri contenuti nella seria.

In `series_works_count` vengono inclusi anche eventuali edizioni speciali, prequels, spin-off etc. Ad esempio, la serie *Harry Potter* è composta da 7 libri principali, ma ha 18 libri in totale in quanto *Goodreads* include il libro prequel, la versione teatrale e varie collezioni speciali.

### Autori
Il file *goodreads_book_authors.csv* contiene i seguenti campi:
- `author_id`: l'id dell'autore;
- `name`: nome dell'autore
- `average_rating`: giudizio medio dei libri;
- `ratings_count`: numero di giudizi ricevuti
- `text_reviews_count`: numero di recensioni ricevute.

### User-Book Interactions
Il file *goodreads_interactions.csv* contiene i seguenti campi:
- `user_id`: l'id dell'utente;
- `book_id`: l'id del libro;
- `is_read`: valore booleano; indica se il libro è stato letto o meno;
- `rating`: il giudizio dato dall'utente, compreso tra 0-5;
- `is_reviewed`: valore booleano; indica se l'utente ha lasciato una recensione o meno.

Poiché il file CSV originale non usa gli stessi `book_id` usati nel file *goodreads_books.csv* è stato caricato anche il file *book_id_map.csv*, che mappa da `book_id` delle interazioni al `book_id` dei libri.

## Preparazione dei dati

Si configura il kernel Spark e si definiscono i percorsi dei vari file salvati sul servizio "Amazon S3", avviando così una nuova applicazione Spark.

In [ ]:
%%configure -f
{
    "executorMemory": "8G", 
    "numExecutors": 2, 
    "executorCores": 3, 
    "conf": {
        "spark.dynamicAllocation.enabled": "false"
    }
}

In [ ]:
val bucketName = "unibo-bd2122-ysun/goodread"

val pathBookDataset = s"s3a://$bucketName/goodreads_books.csv"
val pathSeriesDataset = s"s3a://$bucketName/goodreads_book_series.csv"
val pathAuthorsDataset = s"s3a://$bucketName/goodreads_book_authors.csv"
val pathInteractionsDataset = s"s3a://$bucketName/goodreads_interactions.csv"
val pathBookIdMapDataset = s"s3a://$bucketName/book_id_map.csv"
val pathBookAuthorMapDataset = s"s3a://$bucketName/book_author_map.csv"

"SPARK UI: Enable forwarding of port 20888 and connect to http://localhost:20888/proxy/" + sc.applicationId + "/"